### 화훼 종류 분류

<br>

[주요 화훼류 품질 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=71500)
<br>[화훼 종류 분류](https://aifactory.space/task/2675/overview)

<br>

In [ ]:
import os
import cv2
import sys
import math
import numpy as np
import pandas as pd
from tensorflow.keras.utils import Sequence
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
class Dataloader(Sequence):
    def __init__(self, base_dataset_path, images, labels, batch_size):
        self.base_dataset_path = base_dataset_path
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.indices = np.arange(len(self.labels))

    def __len__(self):
        return math.ceil(len(self.labels)/self.batch_size)

    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_x = [self.images[i] for i in indices]
        batch_images = self.get_imagesets(batch_x)
        batch_images = batch_images.astype('float32') / 255.0

        batch_y = [self.labels[i] for i in indices]
        batch_y = to_categorical(batch_y, 9)
        return np.array(batch_images), np.array(batch_y)

    def get_imagesets(self, path_list):
        image_list = []
        for image in path_list:
            image_path = os.path.join(self.base_dataset_path, image)
            image_list.append(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE))
        return np.array(image_list)


x_train_path = '/content/train'
y_train_path = '/content/train.csv'
model_save_path = 'Flower_classifier.h5'


# 데이터 로드
print('load data')
df = pd.read_csv(y_train_path)
X_train_path, x_test_path, Y_train, y_test = train_test_split(df['image'].values, df['label'].values, train_size=0.8, shuffle=True, stratify=df['label'])

train_loader = Dataloader(x_train_path, X_train_path, Y_train, batch_size=64)
test_loader = Dataloader(x_train_path, x_test_path, y_test, batch_size=64)

# 모델 생성
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(9, activation='softmax')  # num_classes에는 종류별 클래스 수를 설정하세요.
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# 모델 학습
model.fit(train_loader, validation_data=test_loader, epochs=10, batch_size=64)#, validation_data=(x_test, y_test))

# 모델 저장
model.save(model_save_path)

load data
Epoch 1/15
 35/504 [=>............................] - ETA: 51s - loss: 2.0900 - accuracy: 0.2598

In [12]:
import os
import cv2
import sys
import math
import numpy as np
import pandas as pd
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import load_model

class Dataloader(Sequence):
    def __init__(self, base_dataset_path, images, batch_size):
        self.base_dataset_path = base_dataset_path
        self.images = images
        self.batch_size = batch_size
        self.indices = np.arange(len(self.images))

    def __len__(self):
        return math.ceil(len(self.images)/self.batch_size)

    def __getitem__(self, idx):
        indices = self.indices[idx*self.batch_size : (idx+1)*self.batch_size]
        batch_x = [self.images[i] for i in indices]
        batch_images = self.get_imagesets(batch_x)
        batch_images = batch_images.astype('float32') / 255.0
        return np.array(batch_images)

    def get_imagesets(self, path_list):
        image_list = []
        for image in path_list:

            image_path = os.path.join(self.base_dataset_path, image)
            image_list.append(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE))
        return np.array(image_list)


model_path = 'Flower_classifier.h5'
x_test_path = '/content/test'
y_pred_save_path = 'Predict.csv'

image_names = []
labels = []

# 모델 로드
model = load_model(model_path)

df = pd.DataFrame(os.listdir(x_test_path), columns=['image'])
inference_dataloader = Dataloader(x_test_path, df['image'].values, 64)

# 모델 예측
pred = model.predict(inference_dataloader)

pred = list(map(np.argmax, pred))
df['label'] = pred

df.to_csv(y_pred_save_path, index=False)


270/270 [==============================] - 18s 67ms/step
